In [ ]:
!pip install tensorflow==1.15.3
!pip install tensorflow-gpu==1.15.3
!pip install keras==2.2.4
!pip install git+https://github.com/matterport/Mask_RCNN.git
!pip install Pillow
!pip install tqdm

  Cloning https://github.com/matterport/Mask_RCNN.git to /tmp/pip-req-build-qudcndym
  Running command git clone -q https://github.com/matterport/Mask_RCNN.git /tmp/pip-req-build-qudcndym
  Created wheel for mask-rcnn: filename=mask_rcnn-2.1-cp36-none-any.whl size=56922 sha256=b8e082ce00d2b4c0fb9c141ab8e0f5aab44f7010a71c8e7ec2c3fbfdc563c053
  Stored in directory: /tmp/pip-ephem-wheel-cache-87bqfqtx/wheels/c2/db/78/1af79db27d80d68b0eb94d95fda90b8b2c2f6e8862b13d5d01
Successfully built mask-rcnn


In [ ]:
import tensorflow as tf
print(tf.__version__)

tf.test.is_gpu_available()

1.15.3


True

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
DATA_ROOT = '/content/gdrive/My Drive/images'

# Extract pictures
import zipfile
with zipfile.ZipFile(f'{DATA_ROOT}/data/100k_200k_256.zip', 'r') as zip_ref:
    zip_ref.extractall(f'{DATA_ROOT}/data/100k_200k_256')

In [ ]:
from mrcnn.config import Config
from mrcnn.model import MaskRCNN

class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "building_prediction_cfg"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 1
    IMAGES_PER_GPU = 16

model = MaskRCNN(mode='inference', model_dir=f'/{DATA_ROOT}/building_detection/', config=PredictionConfig())
model.load_weights(f'{DATA_ROOT}/mask_rcnn_trained.h5', by_name=True)

Using TensorFlow backend.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








In [ ]:
import os
import numpy as np
import pickle
from PIL import Image
from tqdm import tqdm

# Loop on images and build predictions
detections = {}
batch_nb, batch_images, batch_names = 0, [], []
for image in tqdm(os.listdir(f'{DATA_ROOT}/data/100k_200k_256/256')):
  if image!='.DS_Store':
    img = np.array(Image.open(f'{DATA_ROOT}/data/100k_200k_256/256/{image}'))[None]

    batch_images.append(img)
    batch_names.append(image)
    batch_nb += 1

    if batch_nb == model.config.BATCH_SIZE:
      pred = model.detect(np.concatenate(batch_images))
      for n, p in zip(batch_names, pred):
        detections[n] = p

      batch_nb, batch_images, batch_names = 0, [], []

pickle.dump(detections, open(f'{DATA_ROOT}/data/predictions.pkl', 'wb'))


100%|██████████| 23610/23610 [2:21:47<00:00,  2.78it/s]
